# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv', index_col=0)
weather_df = weather_df.rename(columns={'city': 'City',
                                       'lat': 'Lat',
                                       'lng': 'Lng'})

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lensk,60.7253,114.9278,66.36,89,90,1.81,RU,1659032892
1,new norfolk,-42.7826,147.0587,42.30,84,33,1.99,AU,1659032892
2,lebu,-37.6167,-73.6500,53.71,72,0,12.44,CL,1659032892
3,juba,4.8517,31.5825,71.82,84,62,2.64,SS,1659032893
4,kapaa,22.0752,-159.3190,79.48,80,75,10.36,US,1659032893


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# use the lat and lng as locations and humidity as the weight
locations = weather_df[['Lat', 'Lng']]
weight = weather_df['Humidity'].astype(float)

# add heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False,
                                max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# look for ideal weather condition
    # max temperature between 60 and 80
    # low humidity (maybe less than 60)
    # wind speed less than 10 mph
    # zero to 50% cloudiness

ideal_weather_df = weather_df.loc[(weather_df['Max Temp'] > 60) &
                                 (weather_df['Max Temp'] < 80) &
                                 (weather_df['Wind Speed'] < 10) &
                                 (weather_df['Humidity'] < 60) &
                                 (weather_df['Cloudiness'] < 50)]

ideal_weather_df = ideal_weather_df.dropna(how='any')

In [5]:
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,la ronge,55.1001,-105.2842,73.65,38,20,8.05,CA,1659032921
122,illapel,-31.6308,-71.1653,64.24,50,2,5.21,CL,1659032927
128,kenai,60.5544,-151.2583,64.02,53,0,6.91,US,1659032930
152,thompson,55.7435,-97.8558,73.56,40,20,8.05,CA,1659032687
154,samaipata,-18.1500,-63.8667,71.60,43,28,7.36,BO,1659032939
222,hashtgerd,35.9642,50.6858,78.64,50,16,4.85,IR,1659032964
328,shar,49.5886,81.0453,73.78,48,35,3.02,KZ,1659032993
356,tahara,34.6667,137.2667,78.75,2,17,5.59,JP,1659033003
382,flin flon,54.7682,-101.8650,75.04,33,20,3.44,CA,1659033015
443,erzin,36.9559,36.2000,78.84,58,0,1.92,TR,1659033035


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

## maybe write some code that will drop a row if the resulting coordinates are diff from expected (see thompson)

In [6]:
# create a copy of ideal weather dataframe and name it hotel_df
hotel_df = ideal_weather_df.copy(deep=True).reset_index(drop=True)

# create a column for hotel name
hotel_df['Hotel Name'] = ''

# set parameters for hotel search
target_type = 'lodging'
target_radius = 5000

# create url bases
url_coords = "https://maps.googleapis.com/maps/api/geocode/json"
url_hotels = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for row in range(len(hotel_df)):

    city = hotel_df.iloc[row]['City']
    country = hotel_df.iloc[row]['Country']
    
    location = f'{city}, {country}'
    
    try:
        # get city coordinates
        params = {'address': location, "key": g_key}
        response = requests.get(url_coords, params = params).json()
        
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
        coordinates = f'{lat}, {lng}'
    
        # print coordinates to cross reference
        # print(f"{city}: {lat}, {lng}")
        
        # search for hotels
        parameters = {
            'location': coordinates,
            'radius': target_radius,
            'type': target_type,
            'key': g_key
        }
    
        hotels_res = requests.get(url_hotels, params=parameters).json()
        hotel_df.at[row, 'Hotel Name'] = hotels_res['results'][0]['name']

    
    except:
        print(f'Sorry, but we could not find {location}.')
    

# drop the rows where google couldn't find the city
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df

Sorry, but we could not find shar, KZ.
Sorry, but we could not find taywarah, AF.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,la ronge,55.1001,-105.2842,73.65,38,20,8.05,CA,1659032921,Northland Motor Hotel
1,illapel,-31.6308,-71.1653,64.24,50,2,5.21,CL,1659032927,Hotel Londres
2,kenai,60.5544,-151.2583,64.02,53,0,6.91,US,1659032930,Aspen Extended Stay Suites
3,thompson,55.7435,-97.8558,73.56,40,20,8.05,CA,1659032687,Vino Bello Resort
4,samaipata,-18.1500,-63.8667,71.60,43,28,7.36,BO,1659032939,Casa - Hostal
5,hashtgerd,35.9642,50.6858,78.64,50,16,4.85,IR,1659032964,Parse System
7,tahara,34.6667,137.2667,78.75,2,17,5.59,JP,1659033003,Tahara City Hotel
8,flin flon,54.7682,-101.8650,75.04,33,20,3.44,CA,1659033015,Victoria Inn Flin Flon
9,erzin,36.9559,36.2000,78.84,58,0,1.92,TR,1659033035,Artemis Otel
10,zhangye,38.9342,100.4517,74.01,32,0,7.58,CN,1659033050,Zhangye Lvzhou Holiday Inn


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,
                                info_box_content=hotel_info)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))